In [1]:
import sys
sys.path.append('/Users/jerald/Documents/Dir/Python/stocker')

################################################################################################
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt 
import json 
import yfinance as yf
import sqlite3 as sql
from bin.main import get_path
connections = get_path('../../../')

In [2]:
from alerts.plays.dxp import dxp 

In [3]:
d = dxp(connections)

In [18]:
df = d.run('aapl').reset_index()

In [22]:
expirations = sorted(list(df.expiry.unique()))
exp = expirations[0]

# First 5 Expirations
cry = (df.expiry.isin(expirations[:5]))
gbdf = df[cry].copy().groupby(['expiry','type','strike','sd']).agg({'volume':'sum', 'openinterest':'sum', 'cash':'sum', 'lastprice':'last'})
gbdf.head()

volume  openinterest       cash  lastprice
expiry     type strike sd                                            
2024-11-01 Call 242.5  1     5156          7997   81169.55       1.10
                252.5  2      855          7108  143226.20       0.09
           Put  212.5  2      303          3534   70149.90       0.15
                222.5  1     2621         10690  105296.50       0.73
2024-11-08 Call 242.5  1      776           895    9084.25       2.04

In [23]:
gbdf = d.odf.copy().groupby(['expiry','type']).agg({'volume':'sum', 'openinterest':'sum', 'lastprice':'last'})
gbdf.head()

volume  openinterest  lastprice
expiry     type                                 
2024-11-01 Call   70199        181306       0.01
           Put    51362        137298      66.80
2024-11-08 Call   15183         59104       0.01
           Put    21000         54990      59.55
2024-11-15 Call   22148        401511       0.01

In [100]:
df['play'] = df['type'] + '_' + df['sd'].astype('str')
df['expiry'] = pd.to_datetime(df['expiry'])
df['exp'] = df['expiry'].apply(lambda x: x.strftime('%-m/%-d'))
# Step 1: Get whats priced in 
priced_in = df.set_index(['stock','expiry','exp'])[['stk_price','empct']].drop_duplicates().sort_index().reset_index().to_dict('records')[0]
priced_in['empct2'] = priced_in['empct'] * 2
priced_in

{'stock': 'AAPL',
 'expiry': Timestamp('2024-11-01 00:00:00'),
 'exp': '11/1',
 'stk_price': 232.35,
 'empct': 0.04109102646868948,
 'empct2': 0.08218205293737896}

In [107]:
plays = df[(df.expiry == priced_in['expiry'])].sort_values('sd', ascending=True)[['play','strike','lastprice', 'be']]
plays

,play,strike,lastprice,be
35,Put_1,222.5,0.73,-0.045535
54,Call_1,242.5,1.10,0.048418
18,Put_2,212.5,0.15,-0.086077
75,Call_2,252.5,0.09,0.087110


In [98]:
summary_text = f'''${priced_in['stock']} is trading at ${priced_in['stk_price']} and is expecting a {priced_in['empct']:.2%} move by {priced_in['exp']:}\n'''
play_text = f'''Lottos: \n'''
print(summary_text)

$AAPL is trading at $232.35 and is expecting a 4.11% move by 11/1



0    2027-01-15
1    2025-01-17
2    2024-12-06
3    2025-12-19
4    2026-01-16
        ...    
71   2024-12-06
72   2025-03-21
73   2024-11-15
74   2024-11-08
75   2024-11-01
Name: expiry, Length: 76, dtype: datetime64[ns]